In [1]:
import pandas as pd
import datetime
import logging
from sklearn.model_selection import KFold
import argparse
import json
import numpy as np

import sys
sys.path.append('..')
from utils import load_datasets, load_target
from logs.logger import log_best
from models.lgbm import train_and_predict
import os
os.chdir('../')

%matplotlib inline

In [2]:
os.getcwd()

'C:\\Users\\takuy\\work\\work_tokumoto\\splatoon_competition'

In [5]:
parser = argparse.ArgumentParser()
parser.add_argument('--config', default='./configs/default_optuna.json')
options = parser.parse_args(args=[])
config = json.load(open(options.config))

feats = config['features']
logging.debug(feats)

target_name = config['target_name']

In [6]:
X_train_all, X_test = load_datasets(feats, target_name)
y_train_all = load_target(target_name)

In [7]:
X_train_all.head()

,A1-level,A2-level,A3-level,A4-level,B1-level,B2-level,B3-level,B4-level,enc_period,enc_game-ver,...,enc_A4-weapon,enc_A4-rank,enc_B1-weapon,enc_B1-rank,enc_B2-weapon,enc_B2-rank,enc_B3-weapon,enc_B3-rank,enc_B4-weapon,enc_B4-rank
0,139,118.0,13.0,10.0,28,26.0,68.0,31.0,0.515152,0.525161,...,0.530612,0.554801,0.501116,0.556006,0.570447,0.556006,0.598246,0.556006,0.498008,0.557492
1,198,77.0,198.0,123.0,83,118.0,168.0,151.0,0.466667,0.524499,...,0.542091,0.554401,0.503333,0.555556,0.585821,0.555556,0.592000,0.555556,0.516249,0.556777
2,114,68.0,225.0,107.0,50,163.0,160.0,126.0,0.600000,0.525747,...,0.504202,0.531184,0.493896,0.526357,0.505183,0.530684,0.512195,0.542117,0.529514,0.525678
3,336,131.0,189.0,41.0,273,189.0,194.0,391.0,0.571429,0.523781,...,0.529047,0.552484,0.584475,0.553457,0.579299,0.553457,0.488304,0.553457,0.552743,0.554913
4,299,97.0,96.0,136.0,101,45.0,246.0,160.0,0.580000,0.524329,...,0.536270,0.519280,0.519149,0.518985,0.483444,0.518985,0.513580,0.518985,0.499267,0.518624


In [8]:
# 訓練データとテストデータの列を確認
print(X_train_all.columns)
print(X_test.columns)

Index(['A1-level', 'A2-level', 'A3-level', 'A4-level', 'B1-level', 'B2-level',
       'B3-level', 'B4-level', 'enc_period', 'enc_game-ver', 'enc_lobby-mode',
       'enc_lobby', 'enc_mode', 'enc_stage', 'enc_A1-weapon', 'enc_A1-rank',
       'enc_A2-weapon', 'enc_A2-rank', 'enc_A3-weapon', 'enc_A3-rank',
       'enc_A4-weapon', 'enc_A4-rank', 'enc_B1-weapon', 'enc_B1-rank',
       'enc_B2-weapon', 'enc_B2-rank', 'enc_B3-weapon', 'enc_B3-rank',
       'enc_B4-weapon', 'enc_B4-rank'],
      dtype='object')
Index(['A1-level', 'A2-level', 'A3-level', 'A4-level', 'B1-level', 'B2-level',
       'B3-level', 'B4-level', 'enc_period', 'enc_game-ver', 'enc_lobby-mode',
       'enc_lobby', 'enc_mode', 'enc_stage', 'enc_A1-weapon', 'enc_A1-rank',
       'enc_A2-weapon', 'enc_A2-rank', 'enc_A3-weapon', 'enc_A3-rank',
       'enc_A4-weapon', 'enc_A4-rank', 'enc_B1-weapon', 'enc_B1-rank',
       'enc_B2-weapon', 'enc_B2-rank', 'enc_B3-weapon', 'enc_B3-rank',
       'enc_B4-weapon', 'enc_B4-rank'],
  

In [9]:
# 訓練データとテストデータの列を確認
print(X_train_all.shape)
print(X_test.shape)

(66125, 30)
(28340, 30)


In [10]:
#　訓練データに欠損がないことの確認
X_train_all.isnull().sum().sum()

0

In [11]:
#　テストデータに欠損がないことの確認
X_test.isnull().sum().sum()

0

In [12]:
print("-------------------train-------------------")
print(X_train_all.isnull().sum())
print("-------------------test-------------------")
print(X_test.isnull().sum())

-------------------train-------------------
A1-level          0
A2-level          0
A3-level          0
A4-level          0
B1-level          0
B2-level          0
B3-level          0
B4-level          0
enc_period        0
enc_game-ver      0
enc_lobby-mode    0
enc_lobby         0
enc_mode          0
enc_stage         0
enc_A1-weapon     0
enc_A1-rank       0
enc_A2-weapon     0
enc_A2-rank       0
enc_A3-weapon     0
enc_A3-rank       0
enc_A4-weapon     0
enc_A4-rank       0
enc_B1-weapon     0
enc_B1-rank       0
enc_B2-weapon     0
enc_B2-rank       0
enc_B3-weapon     0
enc_B3-rank       0
enc_B4-weapon     0
enc_B4-rank       0
dtype: int64
-------------------test-------------------
A1-level          0
A2-level          0
A3-level          0
A4-level          0
B1-level          0
B2-level          0
B3-level          0
B4-level          0
enc_period        0
enc_game-ver      0
enc_lobby-mode    0
enc_lobby         0
enc_mode          0
enc_stage         0
enc_A1-weapon     0


In [13]:
display(X_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28340 entries, 0 to 28339
Data columns (total 30 columns):
A1-level          28340 non-null int64
A2-level          28340 non-null float64
A3-level          28340 non-null float64
A4-level          28340 non-null float64
B1-level          28340 non-null int64
B2-level          28340 non-null float64
B3-level          28340 non-null float64
B4-level          28340 non-null float64
enc_period        28340 non-null float64
enc_game-ver      28340 non-null float64
enc_lobby-mode    28340 non-null float64
enc_lobby         28340 non-null float64
enc_mode          28340 non-null float64
enc_stage         28340 non-null float64
enc_A1-weapon     28340 non-null float64
enc_A1-rank       28340 non-null float64
enc_A2-weapon     28340 non-null float64
enc_A2-rank       28340 non-null float64
enc_A3-weapon     28340 non-null float64
enc_A3-rank       28340 non-null float64
enc_A4-weapon     28340 non-null float64
enc_A4-rank       28340 non-null f

None

# modeling

In [14]:
import optuna.integration.lightgbm as lgb
import logging

from logs.logger import log_evaluation


def train_and_predict(X_train, X_valid, y_train, y_valid, X_test, lgbm_params):

    # データセットを生成する
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

    logging.debug(lgbm_params)

    # ロガーの作成
    logger = logging.getLogger('main')
    callbacks = [log_evaluation(logger, period=30)]
    
    best_params, history = {}, []
    # 上記のパラメータでモデルを学習する
    model = lgb.train(
        lgbm_params, lgb_train,
        # モデルの評価用データを渡す
        valid_sets=lgb_eval,
        # 学習過程の表示をoff         
        verbose_eval=False,
        # 最大で 1000 ラウンドまで学習する
        num_boost_round=1000,
        # 10 ラウンド経過しても性能が向上しないときは学習を打ち切る
        early_stopping_rounds=10,
        # 最適なパラメータの採用
        best_params=best_params,
        
        tuning_history=history,
        # ログ
        callbacks=callbacks
    )

    # テストデータを予測する
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    return y_pred, model



In [15]:
y_preds = []
models = []

lgbm_params = config['lgbm_params']

kf = KFold(n_splits=10, random_state=0)
for train_index, valid_index in kf.split(X_train_all):
    X_train, X_valid = (
        X_train_all.iloc[train_index, :], X_train_all.iloc[valid_index, :]
    )
    y_train, y_valid = y_train_all[train_index], y_train_all[valid_index]

    # lgbmの実行
    y_pred, model = train_and_predict(
        X_train, X_valid, y_train, y_valid, X_test, lgbm_params
    )

    # 結果の保存
    y_preds.append(y_pred)
    models.append(model)

    # スコア
    log_best(model, config['loss'])


C:\Users\takuy\AppData\Local\Continuum\anaconda3\envs\dls\lib\site-packages\optuna\_experimental.py:61: ExperimentalWarning:

train is experimental (supported from v0.18.0). The interface can change in the future.

C:\Users\takuy\AppData\Local\Continuum\anaconda3\envs\dls\lib\site-packages\optuna\_experimental.py:83: ExperimentalWarning:

LightGBMTuner is experimental (supported from v1.5.0). The interface can change in the future.

feature_fraction, val_score: 0.684179: 100%|##########| 7/7 [00:05<00:00,  1.35it/s][I 2020-09-26 12:11:46,668] Finished trial#6 with value: 0.6846741502877204 with parameters: {'feature_fraction': 1.0}. Best is trial#5 with value: 0.6841790643887045.

num_leaves, val_score: 0.684179: 100%|##########| 20/20 [00:20<00:00,  1.15it/s][I 2020-09-26 12:12:07,037] Finished trial#26 with value: 0.6857422536613814 with parameters: {'num_leaves': 90}. Best is trial#19 with value: 0.6842793316842355.

bagging, val_score: 0.684179: 100%|##########| 10/10 [00:06<00:00,

In [21]:
# CVスコア
scores = [
    m.best_score['valid_0'][config['loss']] for m in models
]

score = sum(scores) / len(scores)
print('===CV scores===')
print(scores)
print(score)
logging.debug('===CV scores===')
logging.debug(scores)
logging.debug(score)

===CV scores===
[0.6841790417620607, 0.6814048264187383, 0.6829006944550257, 0.6844337962010318, 0.6821577069114579, 0.683854520994051, 0.6848583809620917, 0.683917652353256, 0.6866182760650131, 0.6832872480951787]
0.6837612144217905


In [22]:
y_sub = sum(y_preds) / len(y_preds)

In [23]:
y_sub

array([0.4547118 , 0.48217968, 0.53362444, ..., 0.50240484, 0.46587309,
       0.46356157])

In [27]:
y_preds

[array([0.46541267, 0.46356894, 0.54858079, ..., 0.50651348, 0.45341211,
        0.46636577]),
 array([0.4410475 , 0.52353532, 0.53216233, ..., 0.53549815, 0.41996864,
        0.45682741]),
 array([0.41759539, 0.51191185, 0.54531429, ..., 0.49133615, 0.51110288,
        0.4841149 ]),
 array([0.43993313, 0.48885713, 0.53770904, ..., 0.49019029, 0.51049967,
        0.44731776]),
 array([0.40827327, 0.44894755, 0.54314405, ..., 0.48373486, 0.4503785 ,
        0.43203306]),
 array([0.50506134, 0.44578735, 0.4675175 , ..., 0.47325491, 0.5098232 ,
        0.45790504]),
 array([0.46276521, 0.47323144, 0.51564601, ..., 0.5179085 , 0.42361884,
        0.4897104 ]),
 array([0.46973096, 0.48204628, 0.52879734, ..., 0.50987957, 0.46603293,
        0.46394752]),
 array([0.46787177, 0.49834165, 0.57801889, ..., 0.51834564, 0.4400191 ,
        0.46563394]),
 array([0.46942674, 0.48556933, 0.53935412, ..., 0.49738686, 0.47387505,
        0.47175995])]

In [25]:
sum(y_preds)

array([4.54711799, 4.82179684, 5.33624437, ..., 5.02404841, 4.65873093,
       4.63561575])

In [26]:
len(y_preds)

10